In [ ]:
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
# Importing the dataset
dataset = pd.read_csv('Bank_customer_database.csv')

In [ ]:
#Print some sample rows for investigating the data
dataset.head()

Here the RowNumber, CustomerID and Surname is irrelevent for predicting whether the clinet will exit or not so we just drop these columns (0,1,2). We also split the data into feature vectors X and the target vector y.  

In [ ]:
X = dataset.iloc[:, 3:13].values
y = dataset.iloc[:, 13].values
print("Shape of X", X.shape)
print("Shape of y",y.shape)
print("First rown of X ",X[0,:])
print("First row of y ",y[0])

We notice that the column Geography and Gender are categorical variables so we need to encode them into some integer variables. We do so using LabelEncoder fromt he sklearn.preprocessing package. 

In [ ]:
# Encoding categorical data
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelencoder_X_1 = LabelEncoder()
X[:, 1] = labelencoder_X_1.fit_transform(X[:, 1])
labelencoder_X_2 = LabelEncoder()
X[:, 2] = labelencoder_X_2.fit_transform(X[:, 2])
print("Shape of X", X.shape)
print("Shape of y",y.shape)
print("After encoding the Geography (Column 2 now) and Gender Columns (Column 3 now)")
print("France as 0, Germany as 1, Spain as 2")
print("First row of X ",X[0,:])
print("First row of y ",y[0])

We now need to do OneHotEncoding for the Geography Column 2

In [ ]:
onehotencoder = OneHotEncoder(categorical_features = [1])
X = onehotencoder.fit_transform(X).toarray()
print("Shape of X", X.shape)
print("Shape of y",y.shape)
print("First row of X ",X[0,:])
print("First row of y ",y[0])

Now (1 0 0) is France, (0 1 0) is Germany and (0 0 1) is Spain. We can also exclude the first column since the last two are sufficient.

In [ ]:
X = X[:, 1:]
print("Shape of X", X.shape)
print("Shape of y",y.shape)
print("First row of X ",X[0,:])
print("First row of y ",y[0])

In [ ]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [ ]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

Build and train the ANN

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense
ANN_classifier = Sequential()
ANN_classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))
ANN_classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
ANN_classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
ANN_classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

ANN_classifier.fit(X_train, y_train, batch_size = 10, epochs = 100)

In [ ]:
# Predicting the Test set results
y_pred = ANN_classifier.predict(X_test)
y_pred = (y_pred > 0.5)

In [ ]:
# Predicting a single new observation
"""Predict if the customer with the following informations will leave the bank:
Geography: France
Credit Score: 600
Gender: Male
Age: 40
Tenure: 3
Balance: 60000
Number of Products: 2
Has Credit Card: Yes
Is Active Member: Yes
Estimated Salary: 50000"""
new_prediction = classifier.predict(sc.transform(np.array([[0.0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])))
new_prediction = (new_prediction > 0.5)
print("The above customer leaves ", new_prediction)

In [ ]:
# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print("The confusion matrix is ",cm)

Evaluating  the ANN

In [ ]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from keras.models import Sequential
from keras.layers import Dense
def build_classifier():
    ANN_classifier = Sequential()
    ANN_classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))
    ANN_classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
    ANN_classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    ANN_classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return ANN_classifier
ANN_classifier = KerasClassifier(build_fn = build_classifier, batch_size = 10, epochs = 100)
accuracies = cross_val_score(estimator = ANN_classifier, X = X_train, y = y_train, cv = 10, n_jobs = -1)
mean = accuracies.mean()
variance = accuracies.std()

In [ ]:
print("Mean Accuracy ", mean)
print("Variance ",variance)

Improving the ANN using DropOut

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense
ANN_classifier = Sequential()
ANN_classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))
ANN_classifier.add(Dropout(p=0.1))
ANN_classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
ANN_classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
ANN_classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

ANN_classifier.fit(X_train, y_train, batch_size = 10, epochs = 100)

Hyperparameter tuning using Grid Search

In [ ]:
# Tuning the ANN
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
def build_classifier(optimizer):
    ANN_classifier = Sequential()
    ANN_classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))
    ANN_classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
    ANN_classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    ANN_classifier.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier
ANN_classifier = KerasClassifier(build_fn = build_classifier)
parameters = {'batch_size': [25, 32],
              'epochs': [100, 500],
              'optimizer': ['adam', 'rmsprop']}
grid_search = GridSearchCV(estimator = ANN_classifier,
                           param_grid = parameters,
                           scoring = 'accuracy',
                           cv = 10)
grid_search = grid_search.fit(X_train, y_train)
best_parameters = grid_search.best_params_
best_accuracy = grid_search.best_score_

In [ ]:
print("Best parameters ",best_parameters)
print("Best accuracy ",best_accuracy)